# 1. Pyspark Installation & Config

In [1]:
# %pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# spark.stop()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder\
        .master("local[1]")\
        .appName("FP_BDPAL")\
        .getOrCreate()

spark
sc = spark.sparkContext

22/12/24 15:32:25 WARN Utils: Your hostname, 64-Bit resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp4s0)
22/12/24 15:32:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/24 15:32:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/24 15:32:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# 2. GDrive Mounting

In [4]:
# from google.colab import drive

# drive.mount('/content/drive')

# 3. Importing Data

In [5]:
df_hotelReviews = spark.read.csv('./Hotel_Reviews.csv', header=True, inferSchema=True)

df_hotelReviews.printSchema()

df_hotelReviews.show(5)

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)

+--------------------+----------------------------+-----------+-------------+-----------+--------------------+--------------------+-----------------

P, butuh kagebunshin untuk mengerjakan FP. 2 hari x 9 Matakuliah + 100 menit * 24 sks perminggu

In [6]:
from pyspark.sql import functions as sf

df_reviews = df_hotelReviews.select('Negative_Review', 'Reviewer_Score').rdd.map(lambda x: x['Negative_Review'])
df_reviews.collect()

[' I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it costs the same as a simple double but got way more volume due to the high ceiling was offered a room but only the next day SO i had to check out the next day before 11 o clock in order to get the room i waned to Not the best way to begin 

In [7]:
# df_count = (df_reviews.select('Review').rdd
#             .flatMap(lambda x: x[0].lower().split(' ') ) 
#             .filter(lambda x: len(x)>2)
#             .map(lambda word: (word, 1))
#             .reduceByKey(lambda count1, count2: count1+count2)
#             .map(lambda x: [ 100*(x[1]/N ) ,x[0]])
#             )
# W = df_count.count()

In [8]:
# df_reviews.show()

In [9]:
# df_count.top(50)


In [10]:
%pip install langdetect

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=52d65937db9a331eabe067aaac783b9aed51c4caada0739785de7daac85e3a32
  Stored in directory: /home/user/.cache/pip/wheels/c4/16/af/1889804d8b7c0c041cadee8e29673a938a332acbf2865c70a1
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [19]:
import langdetect as ld

In [20]:
def detect_notEng(review):
    try:
        if ld.detect(review)!= 'en':
            return True
        else:
            return False
    except:
        return True


reviews_notEng = df_reviews.filter(lambda x:len(x)<500).filter(lambda x: detect_notEng(x))

In [21]:
reviews_notEng.collect()

[' n a',
 ' No bad experiences',
 ' Loud aircondition',
 ' Hotel under renovation ',
 ' All ok',
 ' No problems',
 ' n a',
 ' Window and blinds was broken in our room ',
 ' Small room ',
 ' Room building worrk',
 ' bit noisy dark hallways',
 ' Transparentes WC Bad',
 ' A little far from centre breakfast is quite expensive',
 ' One gruff check in member',
 ' No negatives ',
 ' Small rooms ',
 ' NOTHING',
 ' N a',
 ' all good',
 ' no special treatment for regular guests',
 ' ',
 ' Small room',
 ' Little room',
 ' Small room broken tv',
 ' Small rooms',
 ' None',
 ' Our room had a strange smell ',
 ' Rooms were very small',
 ' N A',
 ' N a',
 ' Don t like negatives ',
 ' Expense for meals ',
 ' we didn t dislike anything ',
 ' Nothing ',
 ' hotel needs upgrade',
 ' no bath',
 ' Bedroom was very warm',
 ' N A',
 ' Room hot to sleep in',
 ' No spare blankets Woke up cold due to sudden temperature drop ',
 ' Small rooms ',
 ' No problems at all ',
 ' All good ',
 ' None',
 ' All good ',
 ' N

In [13]:
df_reviews.map(lambda x: x.lower().replace(" i ", " I ")).collect()

[' I am so angry that I made this post available via all possible sites I use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com we stayed for 6 nights in this hotel from 11 to 17 july upon arrival we were placed in a small room on the 2nd floor of the hotel it turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings the room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least I guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that I booked a duplex btw it costs the same as a simple double but got way more volume due to the high ceiling was offered a room but only the next day so I had to check out the next day before 11 o clock in order to get the room I waned to not the best way to begin 

# Installing NLTK

In [14]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

In [17]:
splitRDD_no_stop = df_reviews.filter(lambda x: x.lower() not in stop_words)

splitRDD_no_stop_words = splitRDD_no_stop.map(lambda w: (w, 1))

resultRDD = splitRDD_no_stop_words.reduceByKey(stop_words)

In [18]:
resultRDD.collect()

22/12/24 15:32:43 ERROR Executor: Exception in task 7.0 in stage 5.0 (TID 45)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 540, in func
    return f(iterator)
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 2554, in combineLocally
    merger.mergeVa

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 5.0 failed 1 times, most recent failure: Lost task 5.0 in stage 5.0 (TID 43) (192.168.1.12 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/hom